In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [3]:
# Directory containing soil image datasets categorized into folders
DATA_DIR = 'soils'

In [4]:
# Image size for VGG19
IMG_SIZE = 224

In [5]:
# Function to apply Gabor filters
def apply_gabor_filter(img):
    gabor_kernels = []
    ksize = 31  # kernel size
    for theta in [0, np.pi/4, np.pi/2, 3*np.pi/4]:  # 4 orientations
        kernel = cv2.getGaborKernel((ksize, ksize), 4.0, theta, 10.0, 0.5, 0, ktype=cv2.CV_32F)
        gabor_kernels.append(kernel)
    
    filtered_imgs = [cv2.filter2D(img, cv2.CV_8UC3, k) for k in gabor_kernels]
    return np.mean(filtered_imgs, axis=0).astype(np.uint8)

In [6]:
# Load and preprocess dataset
def load_data():
    X, y = [], []
    class_names = os.listdir(DATA_DIR)
    for idx, label in enumerate(class_names):
        label_dir = os.path.join(DATA_DIR, label)
        for file in os.listdir(label_dir):
            path = os.path.join(label_dir, file)
            img = cv2.imread(path)
            if img is not None:
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                img = apply_gabor_filter(img)
                X.append(img)
                y.append(idx)
    return np.array(X), to_categorical(y), class_names

In [7]:
# Load dataset
X, y, class_names = load_data()
X = X / 255.0  # Normalize
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
# Load VGG19 model without top layer
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
for layer in base_model.layers:
    layer.trainable = False

In [9]:
# Add custom top layers
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(len(class_names), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

In [10]:
# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
# Train model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 175s 11s/step - accuracy: 0.2817 - loss: 1.7487 - val_accuracy: 0.5962 - val_loss: 1.0713
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 221s 14s/step - accuracy: 0.5932 - loss: 1.0548 - val_accuracy: 0.7308 - val_loss: 0.8212
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 98s 6s/step - accuracy: 0.7011 - loss: 0.8158 - val_accuracy: 0.8462 - val_loss: 0.6483
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 224s 15s/step - accuracy: 0.8207 - loss: 0.6071 - val_accuracy: 0.8654 - val_loss: 0.4648
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 141s 9s/step - accuracy: 0.8625 - loss: 0.5094 - val_accuracy: 0.9808 - val_loss: 0.3637
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 165s 11s/step - accuracy: 0.8960 - loss: 0.3703 - val_accuracy: 0.9615 - val_loss: 0.2900
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 113s 7s/step - accuracy: 0.9459 - loss: 0.3250 - val_accuracy: 1.0000 - val_loss: 0.2389
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 108s 7s/step - accuracy: 0.9451 - loss: 0.2955 - val_accuracy: 1.0000 

In [13]:
model.save("soil_classifier_vgg19.h5")

In [14]:
# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

7/7 ━━━━━━━━━━━━━━━━━━━━ 30s 4s/step - accuracy: 0.9731 - loss: 0.2046
Test Accuracy: 96.41%
